# Imports

In [3]:
import matplotlib.pyplot as plt
import numpy
from scipy.optimize import least_squares

%run unit_exponentials.ipynb

# Hyperparameters

In [21]:
# Number of unit exponentials to add
n = 7

# Boundaries of multiplier arguments
min_k = 1.25
max_k = 2.25

# Maximum weight multiplier
r = 3

# Initial value for weights
default_w = 0.5

# Initial value for k_0
default_initial_k = 0.5

# Initial value for other k_j
default_k = 2

# Sums of unit exponentials

In [8]:
# params should be an array [k_0,w_1,k_1,w_2,...,w_n]
def uexp_sum(params, n=n):
    k0 = params[0]
    cur_t = k0
    cur_w = params[1]
    uexps = [uexp(cur_t)]
    ww = [cur_w]

    for i in range(1,n):
        cur_k = params[2*i]
        cur_t = cur_t * cur_k
        uexps.append(uexp(cur_t))
        
        cur_w = params[2*i+1]
        ww.append(cur_w)

    def uexp_sum_f(x):
        result = 0.0
        for i in range(n):
            w = ww[i]
            uexp_f = uexps[i]

            result += w * uexp_f(x)

        return result

    return uexp_sum_f

In [25]:
def uexp_sum_str(params, n=n):
    k0 = params[0]
    cur_t = k0
    cur_w = params[1]

    result = f"{cur_w:.2f}uexp({cur_t:.2f})"

    for i in range(1,n):
        cur_k = params[2*i]
        cur_t = cur_t * cur_k
        cur_w = params[2*i+1]
        result += f" + {cur_w:.2f}uexp({cur_t:.2f})"

    return result

# Least squares fitting

In [22]:
def residuals_absolutely_monotonic_fit_parametric(params, xx, yy, **kwargs):            
    f = uexp_sum(params, **kwargs)

    fxx = f(xx)
    
    return yy - fxx
    
def absolutely_monotonic_fit_parametric(xx, yy, n=n,min_k=min_k,max_k=max_k,default_initial_k=default_initial_k,default_k=default_k,default_w=default_w,r=r,**kwargs):        
    max_y = numpy.max(yy)
    
    min_bounds = [0,0]
    max_bounds = [max_k,r*max_y]
    initial_params = [default_initial_k,default_w]
    
    for i in range(1,n):
        min_bounds.append(min_k)
        min_bounds.append(0)
        max_bounds.append(max_k)
        max_bounds.append(r*max_y)
        initial_params.append(default_k)
        initial_params.append(default_w)
    
    residuals_lambda = lambda params:residuals_absolutely_monotonic_fit_parametric(params,xx,yy,**kwargs)    
    
    result = least_squares(residuals_lambda, initial_params, bounds=(min_bounds,max_bounds))

    return result.x

In [32]:
f = lambda x: numpy.exp(x)-1
xx = numpy.linspace(0,1,100)
yy = f(xx)
print(xx)
print(yy)

result_params = absolutely_monotonic_fit_parametric(xx,yy)
print(result_params)
print(uexp_sum_str(result_params))

[0.         0.01010101 0.02020202 0.03030303 0.04040404 0.05050505
 0.06060606 0.07070707 0.08080808 0.09090909 0.1010101  0.11111111
 0.12121212 0.13131313 0.14141414 0.15151515 0.16161616 0.17171717
 0.18181818 0.19191919 0.2020202  0.21212121 0.22222222 0.23232323
 0.24242424 0.25252525 0.26262626 0.27272727 0.28282828 0.29292929
 0.3030303  0.31313131 0.32323232 0.33333333 0.34343434 0.35353535
 0.36363636 0.37373737 0.38383838 0.39393939 0.4040404  0.41414141
 0.42424242 0.43434343 0.44444444 0.45454545 0.46464646 0.47474747
 0.48484848 0.49494949 0.50505051 0.51515152 0.52525253 0.53535354
 0.54545455 0.55555556 0.56565657 0.57575758 0.58585859 0.5959596
 0.60606061 0.61616162 0.62626263 0.63636364 0.64646465 0.65656566
 0.66666667 0.67676768 0.68686869 0.6969697  0.70707071 0.71717172
 0.72727273 0.73737374 0.74747475 0.75757576 0.76767677 0.77777778
 0.78787879 0.7979798  0.80808081 0.81818182 0.82828283 0.83838384
 0.84848485 0.85858586 0.86868687 0.87878788 0.88888889 0.89898